# Meta-Prompting Example

This notebook demonstrates **Meta-Prompting**, an advanced technique where the AI helps design, refine, and optimize prompts.

**The Concept:**
Instead of manually crafting prompts, you describe your goal to the AI and let it generate an optimized prompt for you. This is particularly useful when:
* You're unsure how to phrase a complex request
* You want to explore different prompting strategies
* You need to adapt prompts for specific use cases

**Key Mechanics:**
1. **Goal Description**: You describe what you want to achieve.
2. **Prompt Generation**: The AI creates an optimized prompt based on best practices.
3. **Iterative Refinement**: You can ask for variations or improvements.
4. **Execution**: Use the generated prompt with your actual task.

In [ ]:
%pip install openai python-dotenv --quiet

### 1. Setup and Authorization

We start by importing the necessary libraries and loading your OpenAI API key.

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    api_key = input("Paste your OpenAI API key: ").strip()

client = OpenAI(api_key=api_key)
print("OpenAI client ready!")

### 2. The Meta-Prompt (The "Prompt Engineer")

This system prompt transforms the AI into a prompt engineering expert. It instructs the model to:
* Understand the user's goal deeply
* Apply prompt engineering best practices
* Generate structured, effective prompts
* Explain the reasoning behind design choices

In [ ]:
META_PROMPT = """
You are an expert Prompt Engineer. Your task is to help users create highly effective prompts.

When a user describes their goal, you will:

1. **Analyze the Goal**: Identify the core task, desired output format, and any constraints.

2. **Apply Best Practices**:
   - Use clear, specific instructions
   - Include relevant context and constraints
   - Specify the desired output format
   - Add examples if helpful (few-shot)
   - Consider edge cases
   - Use appropriate persona/role if beneficial

3. **Generate the Prompt**: Create a complete, ready-to-use prompt wrapped in a code block.

4. **Explain Your Choices**: Briefly explain why you structured the prompt this way.

5. **Offer Variations**: Suggest 1-2 alternative approaches if relevant.

Format your response as:
## Generated Prompt
```
[The optimized prompt here]
```

## Why This Works
[Brief explanation]

## Alternative Approaches
[Optional variations]
"""

### 3. The Meta-Prompting Engine

This function takes your goal and generates an optimized prompt.

**Why use `temperature=0.7`?**
* We want some creativity in prompt design
* But not so much that the output becomes inconsistent
* This balance allows for innovative yet reliable prompt generation

In [ ]:
def generate_prompt(goal: str) -> str:
    """Generate an optimized prompt for the given goal."""
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": META_PROMPT},
                {"role": "user", "content": f"Create an optimized prompt for this goal:\n\n{goal}"}
            ],
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

def refine_prompt(original_prompt: str, feedback: str) -> str:
    """Refine an existing prompt based on feedback."""
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": META_PROMPT},
                {"role": "user", "content": f"""Refine this prompt based on the feedback:

Original Prompt:
{original_prompt}

Feedback:
{feedback}

Generate an improved version."""}
            ],
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

### 4. Interactive Meta-Prompting Session

Now we run the interactive session. Describe your goal and get an optimized prompt!

**Try asking for:**
* "A prompt that summarizes legal documents for non-lawyers"
* "A prompt for generating creative product names"
* "A prompt that helps debug Python code step by step"

In [ ]:
print("Meta-Prompting Engine Ready!")
print("Describe your goal and I'll generate an optimized prompt for you.")
print("Type 'refine' to improve the last prompt, or 'quit' to exit.\n")

last_prompt = None

while True:
    user_input = input("Your goal: ").strip()
    
    if user_input.lower() in ["quit", "exit"]:
        print("Goodbye!")
        break
    
    if not user_input:
        print("Please enter a goal.")
        continue
    
    if user_input.lower() == "refine" and last_prompt:
        feedback = input("What would you like to change? ")
        result = refine_prompt(last_prompt, feedback)
    else:
        result = generate_prompt(user_input)
    
    print(f"\n{result}\n")
    print("-" * 60)
    
    # Extract the prompt for potential refinement
    if "```" in result:
        start = result.find("```") + 3
        end = result.find("```", start)
        if end > start:
            last_prompt = result[start:end].strip()